In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

import pre_processing_funcs as pre

knn = KNeighborsRegressor()

In [2]:
df = pre.set_index(pd.read_csv("data/cleaned_train.csv"))
df.head()

,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,Cost,...,Department,Coffee Bar,Video Store,Bar For Salad,Florist,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
id,,,,,,,,,,,,,,,,,,,,,
mc_ID_0,Dimes Off,Deluxe,8760000.00,4292400.00,True,2842.23,2037.64,481.98,323.00,602.76,...,Household,1,1,1,1,1,28.20,26.60,1.60,10000.00
mc_ID_1,Budget Bargains,Supermarket,6360000.00,1971600.00,False,2814.95,2049.72,457.36,305.02,708.66,...,Snack Foods,0,0,0,0,0,16.57,14.97,1.60,50000.00
mc_ID_2,Shelf Emptiers,Supermarket,10860000.00,4452600.00,True,2192.32,1322.21,523.32,348.85,564.26,...,Periodicals,0,0,0,1,0,28.64,27.18,1.45,30000.00
mc_ID_4,Sale Winners,Deluxe,11560000.00,4970800.00,False,2862.30,1872.19,593.93,395.95,519.76,...,Produce,1,1,1,1,1,12.62,9.71,2.91,50000.00
mc_ID_5,Weekend Discount,Supermarket,5220000.00,1618200.00,True,1970.17,1236.07,440.92,293.95,364.16,...,Household,1,0,0,0,0,15.41,13.95,1.45,30000.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28227 entries, mc_ID_0 to mc_ID_6465
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Promotion Name             28227 non-null  object 
 1   Store Kind                 28227 non-null  object 
 2   Store Sales                28227 non-null  float64
 3   Store Cost                 28227 non-null  float64
 4   Is Recyclable?             28227 non-null  bool   
 5   Store Area                 28227 non-null  float64
 6   Grocery Area               28227 non-null  float64
 7   Frozen Area                28227 non-null  float64
 8   Meat Area                  28227 non-null  float64
 9   Cost                       28227 non-null  float64
 10  Marriage                   28227 non-null  object 
 11  Gender                     28227 non-null  object 
 12  Children                   28227 non-null  int64  
 13  Degree                     28227 non-nul

In [4]:
target = "Cost"
X = df.drop(columns=target)
y = df[target]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
ord_categorical_columns = ['Store Code', 'Promotion Name', 'Store Kind',
                            'Degree', 'Work', 'Order Brand',
                            'Order', 'Department', "Country ISO2"]

hot_categorical_columns = ['Marriage', 'Gender', 'Is Recyclable?']


numeric_columns = X.select_dtypes("float").columns

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), ord_categorical_columns),
        ('ohe', OneHotEncoder(), hot_categorical_columns),
        ('num', StandardScaler(), numeric_columns)
    ]
)

In [8]:
model = make_pipeline(
    preprocessor,
    DecisionTreeRegressor(max_depth=16, random_state=42)
)
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Store Code',
                                                   'Promotion Name',
                                                   'Store Kind', 'Degree',
                                                   'Work', 'Order Brand',
                                                   'Order', 'Department',
                                                   'Country ISO2']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender',
                                                   'Is Recyclable?']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'Net Weight',
       'Package Weight', 'Min. Person Yearly Income'],
      dtype='object'))])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(max_depth=16, random_state=42))])

In [9]:
# Check if the model fitted well or not
predictions = model.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

31.260576325505898

In [10]:
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

63.21910413875263

In [11]:
rfmodel = make_pipeline(
    preprocessor,
    RandomForestRegressor(n_estimators=400, max_depth=16, random_state=42)
)
rfmodel.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Store Code',
                                                   'Promotion Name',
                                                   'Store Kind', 'Degree',
                                                   'Work', 'Order Brand',
                                                   'Order', 'Department',
                                                   'Country ISO2']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender',
                                                   'Is Recyclable?']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'Net Weight',
       'Package Weight', 'Min. Person Yearly Income'],
      dtype='object'))])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=16, n_estimators=400,
                                       random_state=42))])

In [12]:
# Check if the model fitted well or not
predictions = rfmodel.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

29.1118602752378

In [13]:
predictions = rfmodel.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

51.92352323362081

In [14]:
preprocessor.fit(X_train)

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

## Try Lazypredict

In [15]:
reg = LazyRegressor(verbose=1,ignore_warnings=False, custom_metric=None)
models,predictions = reg.fit(X_train_transformed, X_test_transformed, y_train, y_test)

  2%|▏         | 1/42 [00:00<00:34,  1.19it/s]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.11951398275584835, 'Adjusted R-Squared': 0.11559723001010036, 'RMSE': 148.32596671558832, 'Time taken': 0.838735818862915}


  5%|▍         | 2/42 [00:03<01:15,  1.88s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.8789710935965422, 'Adjusted R-Squared': 0.8784327087815802, 'RMSE': 54.992126062592675, 'Time taken': 2.616075277328491}


  7%|▋         | 3/42 [00:03<00:44,  1.14s/it]

...

{'Model': 'TweedieRegressor', 'R-Squared': 0.01137724330900114, 'Adjusted R-Squared': 0.006979455245429089, 'RMSE': 157.17058055162107, 'Time taken': 0.26117563247680664}


 98%|█████████▊| 41/42 [07:14<00:07,  7.77s/it]

{'Model': 'XGBRegressor', 'R-Squared': 0.8769225432878411, 'Adjusted R-Squared': 0.876375045704602, 'RMSE': 55.45557497258384, 'Time taken': 2.4443063735961914}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2516
[LightGBM] [Info] Number of data points in the train set: 22581, number of used features: 25
[LightGBM] [Info] Start training from score 518.858765


100%|██████████| 42/42 [07:15<00:00, 10.37s/it]

{'Model': 'LGBMRegressor', 'R-Squared': 0.8514437425868504, 'Adjusted R-Squared': 0.8507829051428417, 'RMSE': 60.92582667521762, 'Time taken': 0.46050357818603516}


In [16]:
print(models)

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
ExtraTreesRegressor                          0.90       0.90  50.07   
RandomForestRegressor                        0.89       0.89  52.30   
BaggingRegressor                             0.88       0.88  54.99   
XGBRegressor                                 0.88       0.88  55.46   
LGBMRegressor                                0.85       0.85  60.93   
HistGradientBoostingRegressor                0.85       0.85  62.02   
DecisionTreeRegressor                        0.81       0.81  68.99   
ExtraTreeRegressor                           0.71       0.71  84.84   
GradientBoostingRegressor                    0.49       0.49 112.89   
AdaBoostRegressor                            0.12       0.12 148.33   
MLPRegressor                                 0.11       0.11 148.91   
SVR                                          0.03       0.03 155.66   
NuSVR 

In [17]:
df = pd.read_csv("data/train_transformed.csv")
test = pd.read_csv("data/test_transformed.csv")

In [18]:
exmodel = make_pipeline(
    preprocessor,
    ExtraTreesRegressor(n_estimators=500, max_depth=16, random_state=42)
)
exmodel.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Store Code',
                                                   'Promotion Name',
                                                   'Store Kind', 'Degree',
                                                   'Work', 'Order Brand',
                                                   'Order', 'Department',
                                                   'Country ISO2']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender',
                                                   'Is Recyclable?']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'Net Weight',
       'Package Weight', 'Min. Person Yearly Income'],
      dtype='object'))])),
                ('extratreesregressor',
                 ExtraTreesRegressor(max_depth=16, n_estimators=500,
                                     random_state=42))])

In [19]:
# Check if the model fitted well or not
predictions = exmodel.predict(X_train)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

25.296651086740656

In [20]:
predictions = exmodel.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

49.56561217782004

In [21]:
params = {
    'n_estimators': range(100, 301, 100),
    'max_depth': [2, 8, 16, 32, 64],
    'min_samples_split': [2,4],
    'min_samples_leaf': [1,2],
}

gsCV = GridSearchCV(
    ExtraTreesRegressor(random_state=42),
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=1
)
gsCV

GridSearchCV(cv=5, estimator=ExtraTreesRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 8, 16, 32, 64],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': range(100, 301, 100)},
             verbose=1)

In [22]:
gsCV.fit(X_train_transformed, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5, estimator=ExtraTreesRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [2, 8, 16, 32, 64],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': range(100, 301, 100)},
             verbose=1)

In [23]:
gsCV.best_params_

{'max_depth': 16,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 300}

In [24]:
best = gsCV.best_estimator_

In [25]:
# Check if the model fitted well or not
predictions = best.predict(X_train_transformed)

mse = mean_squared_error(y_train, predictions)
rmse = np.sqrt(mse)
rmse

26.8816347915983

In [26]:
predictions = best.predict(X_test_transformed)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rmse

49.676162906712754

In [27]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

kf=KFold(n_splits=5,shuffle=True,random_state=42)

rmse_scorer = make_scorer(rmse, greater_is_better=False)

scores=cross_val_score(exmodel,X,y,cv=kf,scoring=rmse_scorer)

positive_scores = -scores

positive_scores

array([49.70247498, 50.51184942, 50.72072477, 47.99071331, 50.0463652 ])

In [62]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import xgboost as xgb
model = xgb.XGBRegressor()

In [63]:
model = make_pipeline(preprocessor, xgb.XGBRegressor())
model.fit(X_train, y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Store Code',
                                                   'Promotion Name',
                                                   'Store Kind', 'Degree',
                                                   'Work', 'Order Brand',
                                                   'Order', 'Department',
                                                   'Country ISO2']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender',
                                                   'Is Recyclable?']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Froze...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [64]:
y_pred = model.predict(X_test)

In [65]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

Root Mean Squared Error: 55.45


In [66]:
param_grid = {
    'xgbregressor__learning_rate': [0.01, 0.1],
    'xgbregressor__n_estimators': [50, 100],
    'xgbregressor__num_leaves': [31, 63],
}


In [70]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

model = xgb.XGBRegressor(random_state=42)

grid_search = GridSearchCV(
    model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [74]:
grid_search.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


ValueError: 
All the 40 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "E:\ana\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\ana\Lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "E:\ana\Lib\site-packages\xgboost\sklearn.py", line 988, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\ana\Lib\site-packages\xgboost\sklearn.py", line 448, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
                    ^^^^^^^^^^^^^^^
  File "E:\ana\Lib\site-pac

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

In [ ]:
print(f'Best hyperparameters: {grid_search.best_params_}')

In [35]:
import lightgbm as lgb

In [36]:
model = make_pipeline(preprocessor, lgb.LGBMRegressor())
model.fit(X_train, y_train)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001648 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2501
[LightGBM] [Info] Number of data points in the train set: 22581, number of used features: 25
[LightGBM] [Info] Start training from score 518.858765


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['Store Code',
                                                   'Promotion Name',
                                                   'Store Kind', 'Degree',
                                                   'Work', 'Order Brand',
                                                   'Order', 'Department',
                                                   'Country ISO2']),
                                                 ('ohe', OneHotEncoder(),
                                                  ['Marriage', 'Gender',
                                                   'Is Recyclable?']),
                                                 ('num', StandardScaler(),
                                                  Index(['Store Sales', 'Store Cost', 'Store Area', 'Grocery Area',
       'Frozen Area', 'Meat Area', 'Gross Weight', 'Net Weight',
       'Package Weight', 'Min. Person Yearly Income'],
      dtype='object'))])),
                ('lgbmregressor', LGBMRegressor())])

In [37]:
y_pred = model.predict(X_test)

In [38]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

Root Mean Squared Error: 60.93


In [39]:
param_grid = {
    'lgbmregressor__learning_rate': [0.01, 0.1],
    'lgbmregressor__n_estimators': [50, 100],
    'lgbmregressor__num_leaves': [31, 63],
}

In [40]:
grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)

In [41]:
grid_search.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001379 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2498
[LightGBM] [Info] Number of data points in the train set: 18064, number of used features: 25
[LightGBM] [Info] Start training from score 519.190876
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001301 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2503
[LightGBM] [Info] Number of data points in the train set: 18065, number of used features: 25
[LightGBM] [Info] Start training from score 519.171909
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000313 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2502
[LightGBM] [Info] Number of data points in 

In [42]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse:.2f}')

Root Mean Squared Error: 55.35


In [43]:
print(f'Best hyperparameters: {grid_search.best_params_}')

Best hyperparameters: {'lgbmregressor__learning_rate': 0.1, 'lgbmregressor__n_estimators': 100, 'lgbmregressor__num_leaves': 63}


## Extract submissions

In [88]:
samples = pd.read_csv('data/sample_submission.csv')
samples.head()

,ID,Cost
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [89]:
test = pd.read_csv("data/cleaned_test.csv")
test.head()

,id,Promotion Name,Store Kind,Store Sales,Store Cost,Is Recyclable?,Store Area,Grocery Area,Frozen Area,Meat Area,...,Department,Video Store,Bar For Salad,Florist,Coffee Bar,Ready Food,Gross Weight,Net Weight,Package Weight,Min. Person Yearly Income
0,0,Fantastic Discounts,Gourmet,11760000.00,4704000.00,True,2201.06,1424.85,465.54,308.73,...,Snacks,1,1,1,1,1,31.83,28.78,3.05,50000.00
1,1,Bag Stuffer,Deluxe,2160000.00,669600.00,False,2577.16,1735.17,505.07,336.59,...,Frozen Foods,1,1,1,1,1,29.94,27.04,2.91,70000.00
2,2,Pick Your Savings,Deluxe,1830000.00,823500.00,False,2837.58,2038.11,481.98,321.26,...,Dairy,1,1,1,1,1,29.22,26.31,2.91,130000.00
3,3,Price Winners,Deluxe,8820000.00,4410000.00,False,2859.04,1871.16,593.93,394.58,...,Frozen Foods,1,1,1,1,1,28.05,25.00,3.05,10000.00
4,4,Dollar Days,Supermarket,4320000.00,1987200.00,False,2193.97,1320.15,523.32,348.85,...,Beverages,0,0,1,0,0,23.55,20.64,2.91,30000.00


In [90]:
test.isna().sum()

id                           0
Promotion Name               0
Store Kind                   0
Store Sales                  0
Store Cost                   0
Is Recyclable?               0
Store Area                   0
Grocery Area                 0
Frozen Area                  0
Meat Area                    0
Marriage                     0
Gender                       0
Children                     0
Degree                       0
Work                         0
Store Code                   0
Country ISO2                 0
Oreder Brand                 0
Order                        0
Department                   0
Video Store                  0
Bar For Salad                0
Florist                      0
Coffee Bar                   0
Ready Food                   0
Gross Weight                 0
Net Weight                   0
Package Weight               0
Min. Person Yearly Income    0
dtype: int64

In [92]:
y_sub_pred = best.predict(preprocessor.transform(test))

In [93]:
samples["Cost"] = y_sub_pred

In [94]:
samples.to_csv("data/submission.csv", index=False)